In [1]:
import pandas as pd
import numpy as np
import os
import glob
from sharedcontrolpaper.force_sensitive_stopping_task_utils import get_subject_label, aggregate_trial_data, process_trial_data, find_sum_of_intervals, convert_dict_to_df, calculate_proportions_non_nan, collect_trial_metric

In [2]:
EXCLUSIONS = {"s027": ["AI", 80, 96]}

In [3]:
parent_directory = os.path.dirname(os.getcwd())
data_path = os.path.join(parent_directory, 'data', 'experiment')
task = "force_sensitive_stopping_task"
exp_stage = "final"
pattern = os.path.join(data_path, exp_stage, '*', task, '*.csv')
data_files = glob.glob(pattern)

In [4]:
shared_control_metrics = {}
for file in data_files:
    subject_label = get_subject_label(file)
    df = pd.read_csv(file)
    df['block'] = df['block'].str.strip("'")
    df_test = df.query("block != 'practice'")
    block_1 = df_test.query("block == 'block 1'").reset_index(drop=True)
    block_2 = df_test.query("block == 'block 2'").reset_index(drop=True)

    task_dfs = [block_1, block_2]

    for df in task_dfs:
        if 'AI-assisted' in df['condition'].values:
            ai_data = df.copy()
            ai_data_agg = aggregate_trial_data(ai_data)
        else:
            non_ai_data = df.copy()
            non_ai_data_agg = aggregate_trial_data(non_ai_data)


    shared_control_metrics[subject_label] = {'AI': {'data': ai_data_agg}, 'Non-AI': {'data': non_ai_data_agg}}

    for block in shared_control_metrics[subject_label].keys():
        trial_results, ssrt_list = process_trial_data(shared_control_metrics[subject_label][block]['data'], block=block)
        shared_control_metrics[subject_label][block]['trial_results'] = trial_results
        shared_control_metrics[subject_label][block]['ssrt_list'] = ssrt_list

In [5]:
%store shared_control_metrics

Stored 'shared_control_metrics' (dict)


## Excluded Observations

## Grabbing SSRT and other metrics across conditions

In [6]:
def grab_mean_metric(measure, aggregate_ai=False):
    """Calculates subject-level means for a specified metric."""

    condition_measure = {}
    for subject, subject_data in shared_control_metrics.items():
        #Check for exclusions, skip subject if needed
        if subject in EXCLUSIONS and any(trial in EXCLUSIONS[subject] for block in subject_data for trial in subject_data[block]['trial_results']):
            continue

        results = collect_trial_metric(subject_data, measure, aggregate_ai)
        condition_measure[subject] = {
            'non_ai': np.nanmean(results['non_ai']),
            'ai_failed': np.nanmean(results['ai_failed']),
            'ai_assisted': np.nanmean(results['ai_assisted']),
        }
    return pd.DataFrame(condition_measure).T.sort_index()

In [7]:
force_sensitive_stopping_task_ssrt = grab_mean_metric('ssrt')
%store force_sensitive_stopping_task_ssrt

Stored 'force_sensitive_stopping_task_ssrt' (DataFrame)


In [8]:
duration_of_inhibition = grab_mean_metric('duration_of_inhibition')
%store duration_of_inhibition

Stored 'duration_of_inhibition' (DataFrame)


In [9]:
go_task_accuracy_before_stop_onset = grab_mean_metric('go_task_accuracy_before_stop_onset')
%store go_task_accuracy_before_stop_onset

Stored 'go_task_accuracy_before_stop_onset' (DataFrame)


In [10]:
go_task_accuracy_after_stop_onset = grab_mean_metric('go_task_accuracy_after_stop_onset')
%store go_task_accuracy_after_stop_onset

Stored 'go_task_accuracy_after_stop_onset' (DataFrame)


In [11]:
ball_after_ring_proportion_before_stop_onset = grab_mean_metric('ball_after_ring_proportion_before_stop_onset')
print(ball_after_ring_proportion_before_stop_onset.mean())

non_ai         0.000181
ai_failed      0.000095
ai_assisted    0.000241
dtype: float64


In [12]:
first_non_zero_pressure_timestamp = grab_mean_metric('first_non_zero_pressure_timestamp', aggregate_ai=True)
print(first_non_zero_pressure_timestamp.mean())

non_ai         0.051781
ai_failed      0.050405
ai_assisted    0.050405
dtype: float64


In [13]:
first_full_pressure_timestamp = grab_mean_metric('first_full_pressure_timestamp', aggregate_ai=True)
print(first_full_pressure_timestamp.mean())

non_ai         0.179746
ai_failed      0.148457
ai_assisted    0.148457
dtype: float64


In [14]:
proportion_stops_before_stop_onset = grab_mean_metric('proportion_stops_before_stop_onset')
print(proportion_stops_before_stop_onset.mean())

non_ai         0.026004
ai_failed      0.022591
ai_assisted    0.023667
dtype: float64


## Finding the proportion of full pressure points (pressure = 1) at each time interval

In [15]:
non_ai, ai_failed, ai_assisted = {}, {}, {}
for subject, subject_data in shared_control_metrics.items():
    #Check for exclusions, skip subject if needed
    if subject in EXCLUSIONS and any(trial in EXCLUSIONS[subject] for block in subject_data for trial in subject_data[block]['trial_results']):
        continue

    results = collect_trial_metric(subject_data, 'pressures_at_intervals_until_stop_onset')
    # Find the maximum interval length to pad the lists to the same length
    max_length = max(
        max([len(lst) for lst in results['non_ai']], default=0),
        max([len(lst) for lst in results['ai_failed']], default=0),
        max([len(lst) for lst in results['ai_assisted']], default=0)
    )
            
    non_ai = find_sum_of_intervals(results['non_ai'], non_ai, max_length, subject)
    ai_failed = find_sum_of_intervals(results['ai_failed'], ai_failed, max_length, subject)
    ai_assisted = find_sum_of_intervals(results['ai_assisted'], ai_assisted, max_length, subject)


# Convert dictionaries into DataFrames for each condition
time_intervals = [f"{i * 100}-{(i + 1) * 100}ms" for i in range(max_length)]

non_ai_proportion_ones, ai_failed_proportion_ones, ai_assisted_proportion_ones = (convert_dict_to_df(non_ai, time_intervals), 
                                                           convert_dict_to_df(ai_failed, time_intervals), 
                                                           convert_dict_to_df(ai_assisted, time_intervals))

/Users/kritiaxh/Documents/paperRepos/SharedControlPaper/src/sharedcontrolpaper/force_sensitive_stopping_task_utils.py:310: RuntimeWarning: invalid value encountered in divide
  measures_dict[subject] = np.nansum(np.vstack(trials) == 1, axis=0) / counts # Count number of pressures=1


In [16]:
%store non_ai_proportion_ones
%store ai_failed_proportion_ones
%store ai_assisted_proportion_ones

Stored 'non_ai_proportion_ones' (DataFrame)
Stored 'ai_failed_proportion_ones' (DataFrame)
Stored 'ai_assisted_proportion_ones' (DataFrame)


## Finding the proportion of trials where subjects inhibited

In [17]:
condition_measure = {}
proportion = {}

for subject, subject_data in shared_control_metrics.items():
    # Check for exclusions
    if subject in EXCLUSIONS and any(trial in EXCLUSIONS[subject] for block in subject_data for trial in subject_data[block]['trial_results']):
        continue

    results = collect_trial_metric(subject_data, 'ssrt')
    counts, total_counts = calculate_proportions_non_nan(results)


    proportion[subject] = {
        'non_ai': counts['non_ai'] / total_counts['non_ai'] if total_counts['non_ai'] > 0 else 0,
        'ai_failed': counts['ai_failed'] / total_counts['ai_failed'] if total_counts['ai_failed'] > 0 else 0,
        'ai_assisted': counts['ai_assisted'] / total_counts['ai_assisted'] if total_counts['ai_assisted'] > 0 else 0,
    }


df = pd.DataFrame(proportion).T
df = df.sort_index()
print(df.mean())


non_ai         0.999744
ai_failed      0.998718
ai_assisted    0.991667
dtype: float64


## Create CSVs of SSRT by each half of trials in a block 

In [18]:
def grab_mean_metric_by_halves(measure):
    """
    Function to find the mean of a specified metric across different trial conditions 
    for each subject, split by halves of trials.

    Parameters:
    - measure (str): The name of the measure to calculate the mean for.

    Outputs:
    - Saves two CSV files with means for each subject across non_ai trials and ai condition trials
      split by halves.
    """
    # Initialize structures for data collection
    condition_measure_first_half = {}
    condition_measure_second_half = {}

    for subject in shared_control_metrics.keys():
        non_ai_first_half = []
        non_ai_second_half = []
        ai_failed_first_half = []
        ai_failed_second_half = []
        ai_assisted_first_half = []
        ai_assisted_second_half = []
        
        for block in shared_control_metrics[subject].keys():
            trial_results = shared_control_metrics[subject][block]['trial_results']
            num_trials = len(trial_results)

            for index, trial in enumerate(trial_results.keys()):
                if subject in EXCLUSIONS.keys() and trial in EXCLUSIONS[subject] and block in EXCLUSIONS[subject]:
                    continue
                
                ssrt_value = shared_control_metrics[subject][block]['trial_results'][trial][measure]
                
                if pd.isna(ssrt_value):
                    continue
                
                if block == 'Non-AI':
                    if index < num_trials / 2:  # First half
                        non_ai_first_half.append(ssrt_value)
                    else:  # Second half
                        non_ai_second_half.append(ssrt_value)
                    
                elif block == 'AI':
                    condition = shared_control_metrics[subject][block]['trial_results'][trial]['condition']
                    if condition == 'AI-failed':
                        if index < num_trials / 2:  # First half
                            ai_failed_first_half.append(ssrt_value)
                        else:  # Second half
                            ai_failed_second_half.append(ssrt_value)
                    elif condition == 'AI-assisted':
                        if index < num_trials / 2:  # First half
                            ai_assisted_first_half.append(ssrt_value)
                        else:  # Second half
                            ai_assisted_second_half.append(ssrt_value)
                            
        condition_measure_first_half[subject] = {
            'non_ai': np.nanmean(non_ai_first_half),
            'ai_failed': np.nanmean(ai_failed_first_half),
            'ai_assisted': np.nanmean(ai_assisted_first_half)
        }
        
        condition_measure_second_half[subject] = {
            'non_ai': np.nanmean(non_ai_second_half),
            'ai_failed': np.nanmean(ai_failed_second_half),
            'ai_assisted': np.nanmean(ai_assisted_second_half)
        }
    
    df_first_half = pd.DataFrame(condition_measure_first_half).T
    df_second_half = pd.DataFrame(condition_measure_second_half).T

    df_first_half = df_first_half.sort_index()

    df_second_half = df_second_half.sort_index()
    
    return df_first_half, df_second_half

In [19]:
ssrt_first_half, ssrt_second_half = grab_mean_metric_by_halves('ssrt')
%store ssrt_first_half
%store ssrt_second_half

Stored 'ssrt_first_half' (DataFrame)
Stored 'ssrt_second_half' (DataFrame)
